In [38]:
import pandas as pd
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from torch.nn.utils.rnn import pad_sequence
from torch.optim import AdamW

In [39]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## Идея

Судя по тому, что в задании выделена фраза "по эмбеддингу пользователя искать похожие эмбеддинги фильмов", задача сводится к построению таких эмбеддингов, чтобы косинусное сходство между эмбеддингами пользователей и фильмов, которые им нравятся, было максимальным.

Для фильмов нам даны их жанры, теги, проставленные пользователями, а также другой набор тегов с коэффициентами их релевантности для каждого фильма. Всю эту информацию можно использовать для построения эмбеддингов фильмов.
Для пользователей нам даны их индексы, проставленные теги и оценки. По индексам можно строить эмбеддингов, по оценкам - считать ошибку, а с тегами сложнее. Пользователь может ставить теги совершенно разным фильмам, в том числе и тем, которые ему не понравились. Поэтому проставленные пользователем теги не дадут никакой информации о его предпочтениях.

В итоге получаем следующий пайплайн. Строим эмбеддинги фильмов по всей информации, которая у нас о них есть, строим эмбеддинги по пользовательским индексам, вычисляем косинусное сходство между полученными эмбеддингами, и считаем ошибку, используя косинусное сходство и поставленную оценку.

## Предобработка

In [40]:
tags = pd.read_csv("ml-latest-small/tags.csv")
movies = pd.read_csv("ml-latest-small/movies.csv", index_col="movieId")
ratings = pd.read_csv("ml-latest-small/ratings.csv")

Для обработки пользовательский тегов проведем лемматизацию и создадим словарь индексов. Так как большинство тегов однословны, нет смысла проводить какую либо токенизацию или использовать предобученную токенизацию.

In [41]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


Удалим пропущенные теги

In [42]:
tags.isna().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

In [43]:
tags.dropna(inplace=True)

In [44]:
tags.isna().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

In [45]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


Переводим колонку тегов из типа object в строки, приводим к нижнему регистру и применяем токенизацию. Затем создаём словарь тегов и сопоставляем с индексами. Оставим нулевой индекс для случаев, когда тег к фильму будет отсутствовать.

In [46]:
from nltk.stem.snowball import SnowballStemmer

In [47]:
stemmer = SnowballStemmer(language="english")
tags["tag"] = tags["tag"].astype(str).apply(lambda x: stemmer.stem(x.lower()))

In [48]:
unique_tags = tags["tag"].unique()
tags_to_idx = {t: i+1 for i, t in enumerate(unique_tags)}
tags["tag"] = tags["tag"].map(tags_to_idx)
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,1,1445714994
1,2,60756,2,1445714996
2,2,60756,3,1445714992
3,2,89774,4,1445715207
4,2,89774,5,1445715200


Сохраним результат в pd.Series, где каждому индексу фильма будет соответстовать набор индексов пользовательских тегов.

In [49]:
tags_concatenated = tags.groupby("movieId")["tag"].agg(list)

In [50]:
tags_concatenated.head()

movieId
1       [58, 58, 1181]
2    [31, 32, 33, 543]
3           [368, 369]
5           [544, 545]
7                [545]
Name: tag, dtype: object

У фильмов может быть несколько одинаковых тегов, но не будем это исправлять. Это поможет получить более точное представление фильмов.

Полученные наборы индексов тегов добавим в датафрейм с информацией про фильмы

In [51]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [52]:
movies["genres"] = movies["genres"].apply(lambda x: x.split("|")) #переводим жанры из строк в списки

In [53]:
movies["tags"] = tags_concatenated

In [54]:
movies.isna().sum()

title        0
genres       0
tags      8170
dtype: int64

Там, где теги отсутствуют, проставляем нулевые индексы

In [55]:
movies["tags"] = movies["tags"].apply(lambda x: [0] if not isinstance(x, list) else x)

In [56]:
movies.head()

,title,genres,tags
movieId,,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[58, 58, 1181]"
2,Jumanji (1995),"[Adventure, Children, Fantasy]","[31, 32, 33, 543]"
3,Grumpier Old Men (1995),"[Comedy, Romance]","[368, 369]"
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",[0]
5,Father of the Bride Part II (1995),[Comedy],"[544, 545]"


## Разделение выборки

Разделим выборку, содержащую индексы пользователей и проставленные ими рейтингу на 3 части - обучающую, валидационную и тестовую в пропорции 7:2:1. Так как нам нужно создать эмбеддинги для всех пользователей, то все индексы пользователей должны присутствовать в каждой из выборок. Поэтому сгруппируем датафрейм по их индексами и разделение будем проводить внутри полученных групп. Для того, чтобы приблизиться к реальной ситуации, отсортируем строки внутри каждой из групп по времени, старые данные занесем в обучающую выборку, а новые - в валидационную и тестовую 

In [57]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [58]:
#функция для отделения доли выборки в каждой группе
def train_test_split(group, train_size):
    threshold = int(len(group) * train_size)
    return group.iloc[:threshold]

In [59]:
ratings.sort_values(["userId", "timestamp"], inplace=True) #сортируем сначала по пользователям, затем по времени
                                                           #в порядке возрастания значений
train_ratings = ratings.groupby("userId").apply(train_test_split, 0.7).reset_index(drop=True) #отделяем тренировочные данные
val_test_ratings = pd.concat([ratings, train_ratings]).drop_duplicates(keep=False).reset_index(drop=True) #остаток заносим
                                                                                            #в валидационную и тестовую

In [60]:
train_ratings.head()

,userId,movieId,rating,timestamp
0,1,804,4.0,964980499
1,1,1210,5.0,964980499
2,1,2018,5.0,964980523
3,1,2628,4.0,964980523
4,1,2826,4.0,964980523


In [61]:
#аналогично, отделяем валидационную выборку от тестовой
val_ratings = val_test_ratings.groupby("userId").apply(train_test_split, 2/3).reset_index(drop=True)
test_ratings = pd.concat([val_test_ratings, val_ratings]).drop_duplicates(keep=False).reset_index(drop=True)

In [62]:
val_ratings.head()

,userId,movieId,rating,timestamp
0,1,1030,3.0,964982903
1,1,2033,5.0,964982903
2,1,4006,4.0,964982903
3,1,50,5.0,964982931
4,1,608,5.0,964982931


In [63]:
test_ratings.head()

,userId,movieId,rating,timestamp
0,1,1270,5.0,964983705
1,1,1240,5.0,964983723
2,1,1206,5.0,964983737
3,1,3702,5.0,964983737
4,1,3033,5.0,964983762


In [64]:
train_ratings.shape[0], val_ratings.shape[0], test_ratings.shape[0]

(70312, 20162, 10362)

In [65]:
ratings.shape

(100836, 4)

Получили нужные пропорции

При инициализации также закодируем жанры фильмов. Для жанров не требуется предварительной обработки, так как их мало и они все разные. Их перечень есть в документации датасета, выпишем его сюда и создадим словарь

In [66]:
genres = [
    "(no genres listed)",
    "Action", 
    "Adventure", 
    "Animation", 
    "Children", 
    "Comedy", 
    "Crime", 
    "Documentary", 
    "Drama", 
    "Fantasy", 
    "Film-Noir", 
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
    "IMAX"
]
genres_to_idx = {genre: index for index, genre in enumerate(genres)}

## Создание датасета

Внутри батча будут содержаться все описанные выше признаки. Создавать батч будем следующим образом: по полученному индексу обращаемся к таблице с пользователями, фильмами и оценками, из нее получаем индекс фильма и используем его для получения всех остальных признаков во второй таблице.

Так как мы оперерируем тензорами, то внутри батча будем добавлять паддинги из нулей. Нули как раз подходят, так как до этого мы обозначали ими отсутствие тегов или неуказанный жанр.

In [67]:
class MovieDataset(Dataset):
    def __init__(self, movies_data, user_data):
        self.user_idx = user_data["userId"]
        self.ratings = user_data["rating"]
        self.movie_idx = user_data["movieId"]
        self.tag_idx = movies_data["tags"]
        self.genre_idx = movies_data["genres"].apply(lambda x: [genres_to_idx[genre] for genre in x]) #кодируем жанры по словарю
    
    def __getitem__(self, index):
        user_id = self.user_idx[index]
        rating = self.ratings[index]
        movie_id = self.movie_idx[index]
        tags = torch.Tensor(self.tag_idx[movie_id]).int()
        genres = torch.Tensor(self.genre_idx[movie_id]).int()
        return tags, genres, user_id, rating, index
    
    def __len__(self):
        return self.user_idx.shape[0]
    
    def collate_function(self, batch):
        tags, genres, user_id, rating, index = zip(*batch)
        batched_tags = pad_sequence(tags, batch_first=True, padding_value=0)
        batched_genres = pad_sequence(genres, batch_first=True, padding_value=0)
        user_id = torch.Tensor(user_id).int()
        rating = torch.Tensor(rating).int()
        return {
            "user_id": user_id, 
            "tags": batched_tags,
            "genres": batched_genres, 
            "rating": rating, 
            "index": index}

Создаем даталоадеры, объекты перемешиваем с фиксированной случайностью. Я выбрал размер батча 128, так как он самый оптимальный по скорости на моем компьютере

In [81]:
torch.manual_seed(42)
batch_size = 128

train_dataset = MovieDataset(movies, train_ratings)
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    collate_fn=train_dataset.collate_function,
    shuffle=True,
)

val_dataset = MovieDataset(movies, val_ratings)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=val_dataset.collate_function,
    shuffle=True,
)

test_dataset = MovieDataset(movies, test_ratings)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,
    collate_fn=val_dataset.collate_function,
    shuffle=True,
)

## Обучение

Как было описано в начале, наша модель будет создавать эмбеддинги фильмов по их признакам и эмбеддинги пользователям по индексам. Затем мы будем считать косинусное сходство между двумя полученными эмбеддингами и на его основе вычислять лосс, о котором будет написано дальше.

Для получения эмбеддингов фильмов воспользуемся двумя простыми nn.Embedding(), каждый из которых будет создавать векторные представления пользовательских тегов и жанров соответственно. Далее полученные эмбеддинги сконкатенируем, пропустим через линейный слой, чтобы привести к размерности эмбеддинга пользователя, и вычислим лосс. Эмбеддинги пользователя также будем строить с помощью nn.Embedding().

Можно было попробовать закодировать признаки и по-другому - например, с помощью предобученной модели BERT или обучить свой TransformerEncoder. Но предобученная модель кажется слишком избыточной для такой задачи, так как нам особо не нужны точные смысловые представления тегов и жанров. А трансформерные архитектуры все-таки больше подходят для более длинных последовательностей, и тоже могли оказаться избытычными. В нашем случае наши признаки однословные и ни теги, ни жанры друг с другом не так сильно связаны, чтобы образовывать какой-то контекст.

In [69]:
class MovieModel(nn.Module):
    def __init__(self, tag_vocab_size, genre_vocab_size, user_vocab_size, embedding_dim):
        super(MovieModel, self).__init__()
        
        self.tag_embedder = nn.Embedding(tag_vocab_size, embedding_dim, padding_idx=0) #эмбеддер пользовательских тегов
        self.genre_embedder = nn.Embedding(genre_vocab_size, embedding_dim, padding_idx=0) #эмбеддер жанров
        self.user_embedder = nn.Embedding(user_vocab_size, embedding_dim) #эмбеддер пользовательских индексов
        
        self.fc = nn.Linear(2 * embedding_dim, embedding_dim)
        
    def forward(self, input_batch):
        tag_emb = torch.mean(self.tag_embedder(input_batch["tags"]), dim=1)
        genre_emb = torch.mean(self.genre_embedder(input_batch["genres"]), dim=1)
        
        tag_genre_emb = torch.cat((tag_emb, genre_emb), dim=1) #конкатенируем полученные эмбеддинги
        movie_emb = self.fc(tag_genre_emb) #получаем эмбеддинг фильма
        
        user_emb = self.user_embedder(input_batch["user_id"])
        
        return user_emb, movie_emb

Чтобы получить лосс, сначала посчитаем косинусное сходство между векторами, затем переведем его в диапазон от 0 до 5. Затем вычислим сам лосс - Root mean squared error между оценкой пользователя и полученным значением сходства. Я выбрал RMSE, так как его среднее значение можно использовать в качестве интерпретируемой метрики на валидации - будет понятно, на сколько баллов наша предсказанная оценка будет отклоняться от истинной в среднем.

In [70]:
def compute_loss(user_emb, movie_emb, rating):
    cos_sim = nn.CosineSimilarity(dim=1)
    
    sim_norm = (cos_sim(user_emb, movie_emb) + 1) * 5 #переводим из диапазона [-1, 1] в [0, 5]
    rmse_loss = torch.sqrt(torch.mean((sim_norm - rating) ** 2))
    
    return rmse_loss

In [74]:
embedding_dim = 100 #тоже самое эффективное значение по скорости обучения и сходимости
lr = 1e-5
epochs = 1000

model = MovieModel(
    unique_tags.shape[0]+1, 
    len(genres), 
    ratings["userId"].unique().shape[0]+1, 
    embedding_dim, 
)
model.to(device)

optimizer = AdamW(model.parameters(), lr)

pbar = tqdm(total=len(train_dataloader)*epochs)

num_iter = 1
val_every = 10000 #каждые val_every итераций будем проводить валидацию

#будем смотреть на изменение среднего значения лосса на валидации,
#и когда оно станет слишком незначительным, закончим обучение
last_val_loss = 5
delta = 0.01
finished = False

for epoch in range(epochs):
    for batch in train_dataloader:
        model.train()

        for key, val in batch.items(): #переводим все тензоры на устройство вычислений
            if key != "index":
                batch[key] = val.to(device)

        optimizer.zero_grad()

        user_emb, movie_emb = model(batch)
        loss = compute_loss(user_emb, movie_emb, batch["rating"])

        loss.backward()
        optimizer.step()

        pbar.desc = f"Epoch {epoch+1}/{epochs}. Train loss: {round(loss.item(), 3)}"
        pbar.update()
        
        num_iter += 1
        
        if num_iter % val_every == 0:
            model.eval()
            val_loss = 0
            
            for batch in tqdm(val_dataloader):
                for key, val in batch.items():
                    if key != "index":
                        batch[key] = val.to(device)
                user_emb, movie_emb = model(batch)
                val_loss += compute_loss(user_emb, movie_emb, batch["rating"])
            
            mean_val_loss = val_loss.item() / len(val_dataloader)
            print(f"Validation loss: {round(mean_val_loss, 3)}")
            
            #проверка на раннюю остановку
            if last_val_loss - mean_val_loss < delta:
                print("Training is finished.")
                finished = True
                break
            else:
                last_val_loss = mean_val_loss
                
    if finished:
        break
        
pbar.close()

  0%|          | 0/550000 [00:00<?, ?it/s]

  0%|          | 0/158 [00:00<?, ?it/s]

Validation loss: 1.663


  0%|          | 0/158 [00:00<?, ?it/s]

Validation loss: 1.535


...

Validation loss: 1.051


  0%|          | 0/158 [00:00<?, ?it/s]

Validation loss: 1.041


  0%|          | 0/158 [00:00<?, ?it/s]

Validation loss: 1.033
Training is finished.


Видим, что на валидационной выборке модель в среднем ошибается на один балл. Это не самый лучший результат, хоть и не сильно плохой. На большом датасете качество могло бы получиться выше засчет того, что там больше данных, а также есть фиксированные теги с релевантностью, которые могли бы дать гораздо больше информации о фильмах.

Теперь можно смоделировать реальную ситуацию: возьмем случайный объект из тестовой выборки (shuffle=True) и попытаемся предсказать на нем рейтинг

In [82]:
for batch in test_dataloader:
    for key, val in batch.items():
        if key != "index":
            batch[key] = val.to(device)
    user_emb, movie_emb = model(batch)
    loss = compute_loss(user_emb, movie_emb, batch["rating"])
    print(f"Loss: {loss.item()}, true rating: {batch['rating']}")
    break

Loss: 1.2412543296813965, true rating: tensor([3], device='cuda:0', dtype=torch.int32)


Модель предсказала либо 2, либо 4, что в целом не так плохо, как 5 или 0))

Дальше можно для каждого пользователя вычислять косинусное сходство ко всем фильмам, сортировать и выдавать наиболее близкие